# Usage Notes
This notebook should be launched from a session of jupyter notebook that was launched from a DB2 command window
To do this, run an administrator DB2 command window as an administrator and type 'jupyter notebook'

If you don't know what to do right now, try:
1. Update the database connectivity file with your database connectivity information
1. Click on each code sell in the "Environment set up" section and select "Run Cells" from the Cell menu on the toolbar immediately below the name of this notebook.
1.  On the Kernel menu, select "Restart & Run All"

If you have set up your enviornment before or used this Notebook before, then select "Cell" on the toolbar just under the name of this notebook, and select "run all". If database credentials are properly defined, this will generate the snapshot data using values since the last database start.

To emulate monitor reset, click the link that says "Collect baselines (reset monitor switches all)" below

To generate values based on the reset values, click on the text that says "Generate snapshot based on reset values", and then select "Run all Below" on the Cell menu to generate snapshot data based on a previously collected baseline.

# Environment set up:

In [1]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

## Import the modules and load the SQL magic

In [1]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
from IPython.display import HTML
#Import stuff we need
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import numpy as np
import pandas as pd

# All plots should be inline, inside the notebook
%matplotlib inline

In [3]:
## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [2]:
baselines=0
user='db2admin'
host='localhost'
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename
password = LocalDB2password
port=50000
db='SAMPLE'
%sql db2+ibm_db://$user:$password@$host:$port/$db

'Connected: db2admin@SAMPLE'

# Collect Baselines

In [28]:
## Collect Baselines
HTML('''<script>
collect_baseline=false; 
function collect_baselines() {
 var kernel = IPython.notebook.kernel;
 collect_baseline=true;
 if (collect_baseline){
 kernel.execute('baseline_ts = %sql select current timestamp from sysibm.sysdummy1');
 kernel.execute('mgd_baseline = %sql select * from table(mon_get_database(-2)) as mgd');
 kernel.execute('mgtl_baseline = %sql select * from table(mon_get_transaction_log(-2)) as mgtl');
 kernel.execute('mgbp_baseline = %sql select * from table(mon_get_bufferpool(NULL,-2)) mgbp');
 kernel.execute('baselines=1');
 window.alert('Baselines Collected')
 }
} 

</script>
The baselines are not collected by default.
<a href="javascript:collect_baselines()">Collect baselines (reset monitor switches all)</a>.
''')

Generate snapshot based on reset values: Click on this text and then select "Run all Below" on the Cell menu to generate snapshot data based on a previously collected baseline.

In [6]:
## To reset the monitor values, run this cell
## Baseline
#baseline_ts = %sql select current timestamp from sysibm.sysdummy1
#mgd_baseline = %sql select * from table(mon_get_database(-2)) as mgd
#mgtl_baseline = %sql select * from table(mon_get_transaction_log(-2)) as mgtl
#mgbp_baseline = %sql select * from table(mon_get_bufferpool(NULL,-2)) mgbp
#baselines=1

try:
    baseline_ts
except NameError:
    print("No baseline values defined. Data is since last database activation")
else:
    print(baseline_ts)

No baseline values defined. Data is since last database activation


# Db2 Snapshot
Ember's snapshot for Db2 (LUW)

## Instance Uptime

In [7]:
!db2pd -


Database Member 0 -- Active -- Up 4 days 13:16:27 -- Date 2017-07-28-20.37.05.396000



## Database active since

In [8]:
db_conn_time=%sql select db_conn_time from table(mon_get_database(-2)) as mgd
db_conn_time

Done.


db_conn_time
2017-07-27 18:39:16


## Five most recent activation times

In [9]:
## Not working, come back to it
%sql select * \
from table(pd_get_log_msgs(current timestamp - 7 days, -2))\
where DBNAME= :db 


Done.


TIMESTAMP,timezone,instancename,DBPARTITIONNUM,dbname,pid,processname,tid,appl_id,component,FUNCTION,probe,msgnum,msgtype,msgseverity,msg,MEMBER


## Last Succesful Backup

In [10]:
%sql with ts as (select EID from sysibmadm.db_history where start_time = (select max(timestamp(start_time)) as ts  \
		from sysibmadm.db_history \
		where operation='B' and SQLCODE is NULL)\
	) \
select case \
		when operationtype = 'D' then 'DELTA_OFFLINE'\
		when operationtype = 'E' then 'DELTA_ONLINE' \
		when operationtype = 'F' then 'OFFLINE' \
		when operationtype = 'I' then 'INCR_OFFLINE' \
		when operationtype = 'N' then 'ONLINE' \
		when operationtype = 'O' then 'INCR_ONLINE' \
		else operationtype \
	end as type \
        , to_char(timestamp(start_time), 'YYYY-MM-DD HH24:MI:SS')  as backup_time \
FROM sysibmadm.db_history dh \
	join ts on dh.eid = ts.eid \
ORDER BY start_time desc \
fetch first 1 row only

Done.


TYPE,backup_time
ONLINE,2017-04-24 13:01:13


## HADR details

In [11]:
#untested - test on HADR database
%sql select hadr_role \
    , hadr_state \
    , hadr_connect_status \
    , hadr_log_gap \
FROM table(mon_get_hadr(-2))

Done.


hadr_role,hadr_state,hadr_connect_status,hadr_log_gap


## Tablespaces

In [12]:
%sql select count(*) as NUM_TBSPS_NOT_AUTORESIZE \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_AUTO_RESIZE_ENABLED=0 \
    AND TBSP_TYPE='DMS'

Done.


num_tbsps_not_autoresize
0


In [13]:
%sql select tbsp_name \
    , decimal(float(tbsp_used_pages)/float(tbsp_usable_pages),5,2) as pct_full \
FROM table(mon_get_tablespace(NULL,-2)) \
WHERE TBSP_TYPE='DMS' \
    AND (TBSP_AUTO_RESIZE_ENABLED=0 OR TBSP_LAST_RESIZE_FAILED=1)

Done.


tbsp_name,pct_full


## Index Read Efficiency

In [14]:
if baselines == 1:
    read_eff=%sql select decimal((float(rows_read)-{mgd_baseline[0].rows_read})/(float(rows_returned)-{mgd_baseline[0].rows_returned}),16,2) read_eff \
    from table(mon_get_database(-2)) as mgd 
else:
    read_eff=%sql select decimal(float(rows_read)/float(rows_returned),16,2) read_eff \
    from table(mon_get_database(-2)) as mgd
read_eff

Done.


read_eff
25.68


## Buffer Pool Hit Ratios

### Overall

In [15]:
if baselines == 1:
    bp_details = %sql select \
        (pool_data_l_reads-{mgd_baseline[0].pool_data_l_reads} + pool_xda_l_reads-{mgd_baseline[0].pool_xda_l_reads} + pool_index_l_reads-{mgd_baseline[0].pool_index_l_reads}) as total_l_reads \
        , case when pool_data_l_reads-{mgd_baseline[0].pool_data_l_reads} > 0 or pool_temp_data_l_reads-{mgd_baseline[0].pool_temp_data_l_reads} > 0 \
            then decimal(((float(pool_data_lbp_pages_found-{mgd_baseline[0].pool_data_lbp_pages_found}) - float(pool_async_data_lbp_pages_found-{mgd_baseline[0].pool_async_data_lbp_pages_found})) / (float(pool_data_l_reads-{mgd_baseline[0].pool_data_l_reads}) + float(pool_temp_data_l_reads-{mgd_baseline[0].pool_temp_data_l_reads}))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads-{mgd_baseline[0].pool_index_l_reads} > 0 or pool_temp_index_l_reads-{mgd_baseline[0].pool_temp_index_l_reads} > 0 \
            then decimal(((float(pool_index_lbp_pages_found-{mgd_baseline[0].pool_index_lbp_pages_found}) - float(pool_async_index_lbp_pages_found-{mgd_baseline[0].pool_async_index_lbp_pages_found})) / (float(pool_index_l_reads-{mgd_baseline[0].pool_index_l_reads}) + float(pool_temp_index_l_reads-{mgd_baseline[0].pool_temp_index_l_reads}))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads-{mgd_baseline[0].pool_xda_l_reads} > 0 or pool_temp_xda_l_reads-{mgd_baseline[0].pool_temp_xda_l_reads} > 0 \
             then decimal(((float(pool_xda_lbp_pages_found-{mgd_baseline[0].pool_xda_lbp_pages_found}) - float(pool_async_xda_lbp_pages_found-{mgd_baseline[0].pool_async_xda_lbp_pages_found})) / (float(pool_xda_l_reads-{mgd_baseline[0].pool_xda_l_reads}) + float(pool_temp_xda_l_reads-{mgd_baseline[0].pool_temp_xda_l_reads}))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads-{mgd_baseline[0].pool_col_l_reads} > 0 or pool_temp_col_l_reads-{mgd_baseline[0].pool_col_l_reads} > 0 \
             then decimal(((float(pool_col_lbp_pages_found-{mgd_baseline[0].pool_col_lbp_pages_found}) - float(pool_async_col_lbp_pages_found-{mgd_baseline[0].pool_async_col_lbp_pages_found})) / (float(pool_col_l_reads-{mgd_baseline[0].pool_col_l_reads}) + float(pool_temp_col_l_reads-{mgd_baseline[0].pool_temp_col_l_reads}))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_database(-2)) mgd 
else:
    bp_details = %sql select \
        (pool_data_l_reads + pool_xda_l_reads + pool_index_l_reads) as total_l_reads \
        , case when pool_data_l_reads > 0 or pool_temp_data_l_reads > 0 \
            then decimal(((float(pool_data_lbp_pages_found) - float(pool_async_data_lbp_pages_found)) / (float(pool_data_l_reads) + float(pool_temp_data_l_reads))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads > 0 or pool_temp_index_l_reads > 0 \
            then decimal(((float(pool_index_lbp_pages_found) - float(pool_async_index_lbp_pages_found)) / (float(pool_index_l_reads) + float(pool_temp_index_l_reads))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads > 0 or pool_temp_xda_l_reads > 0 \
             then decimal(((float(pool_xda_lbp_pages_found) - float(pool_async_xda_lbp_pages_found)) / (float(pool_xda_l_reads) + float(pool_temp_xda_l_reads))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads > 0 or pool_temp_col_l_reads > 0 \
             then decimal(((float(pool_col_lbp_pages_found) - float(pool_async_col_lbp_pages_found)) / (float(pool_col_l_reads) + float(pool_temp_col_l_reads))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_database(-2)) mgd
bp_details

Done.


total_l_reads,data_hit_ratio_percent,index_hit_ratio_percent,xda_hit_ratio_percent,col_hit_ratio_percent
67042,99.52,97.54,-1.00,-1.00


### By Bufferpool

In [16]:
df = pd.DataFrame()
if baselines == 1:
    for row in mgbp_baseline:
        this_bp=row['bp_name']
        if this_bp.startswith("IBMSYS"):
            #skip this bp, as it is one of the small default ones
            meh=1
        else:
            bp_details = %sql select bp_name \
                , (pool_data_l_reads-{row['pool_data_l_reads']} + pool_xda_l_reads-{row['pool_xda_l_reads']} + pool_index_l_reads-{row['pool_index_l_reads']}) as total_l_reads \
                , case when pool_data_l_reads-{row['pool_data_l_reads']} > 0 or pool_temp_data_l_reads-{row['pool_temp_data_l_reads']} > 0 \
                    then decimal(((float(pool_data_lbp_pages_found-{row['pool_data_lbp_pages_found']}) - float(pool_async_data_lbp_pages_found-{row['pool_async_data_lbp_pages_found']})) / (float(pool_data_l_reads-{row['pool_data_l_reads']}) + float(pool_temp_data_l_reads-{row['pool_temp_data_l_reads']}))) * 100,5,2) \
                    else -1 end as data_hit_ratio_percent \
                , case when pool_index_l_reads-{row['pool_index_l_reads']} > 0 or pool_temp_index_l_reads-{row['pool_temp_index_l_reads']} > 0 \
                    then decimal(((float(pool_index_lbp_pages_found-{row['pool_index_lbp_pages_found']}) - float(pool_async_index_lbp_pages_found-{row['pool_async_index_lbp_pages_found']})) / (float(pool_index_l_reads-{row['pool_index_l_reads']}) + float(pool_temp_index_l_reads-{row['pool_temp_index_l_reads']}))) * 100,5,2) \
                    else -1 end as index_hit_ratio_percent \
                , case when pool_xda_l_reads-{row['pool_xda_l_reads']} > 0 or pool_temp_xda_l_reads-{row['pool_temp_xda_l_reads']} > 0 \
                     then decimal(((float(pool_xda_lbp_pages_found-{row['pool_xda_lbp_pages_found']}) - float(pool_async_xda_lbp_pages_found-{row['pool_async_xda_lbp_pages_found']})) / (float(pool_xda_l_reads-{row['pool_xda_l_reads']}) + float(pool_temp_xda_l_reads-{row['pool_temp_xda_l_reads']}))) * 100,5,2) \
                     else -1 end as xda_hit_ratio_percent \
                , case when pool_col_l_reads-{row['pool_col_l_reads']} > 0 or pool_temp_col_l_reads-{row['pool_col_l_reads']} > 0 \
                     then decimal(((float(pool_col_lbp_pages_found-{row['pool_col_lbp_pages_found']}) - float(pool_async_col_lbp_pages_found-{row['pool_async_col_lbp_pages_found']})) / (float(pool_col_l_reads-{row['pool_col_l_reads']}) + float(pool_temp_col_l_reads-{row['pool_temp_col_l_reads']}))) * 100,5,2) \
                     else -1 end as col_hit_ratio_percent \
            from table(mon_get_bufferpool(NULL,-2)) mgbp \
            WHERE mgbp.bp_name not like 'IBMSYS%' \
                and mgbp.bp_name=:this_bp
        df = df.append(bp_details)
        bp_details=None
else:
    bp_details = %sql select mgbp.bp_name \
        , (pool_data_l_reads + pool_xda_l_reads + pool_index_l_reads) as total_l_reads \
        , case when pool_data_l_reads > 0 or pool_temp_data_l_reads > 0 \
            then decimal(((float(pool_data_lbp_pages_found) - float(pool_async_data_lbp_pages_found)) / (float(pool_data_l_reads) + float(pool_temp_data_l_reads))) * 100,5,2) \
            else -1 end as data_hit_ratio_percent \
        , case when pool_index_l_reads > 0 or pool_temp_index_l_reads > 0 \
            then decimal(((float(pool_index_lbp_pages_found) - float(pool_async_index_lbp_pages_found)) / (float(pool_index_l_reads) + float(pool_temp_index_l_reads))) * 100,5,2) \
            else -1 end as index_hit_ratio_percent \
        , case when pool_xda_l_reads > 0 or pool_temp_xda_l_reads > 0 \
             then decimal(((float(pool_xda_lbp_pages_found) - float(pool_async_xda_lbp_pages_found)) / (float(pool_xda_l_reads) + float(pool_temp_xda_l_reads))) * 100,5,2) \
             else -1 end as xda_hit_ratio_percent \
        , case when pool_col_l_reads > 0 or pool_temp_col_l_reads > 0 \
             then decimal(((float(pool_col_lbp_pages_found) - float(pool_async_col_lbp_pages_found)) / (float(pool_col_l_reads) + float(pool_temp_col_l_reads))) * 100,5,2) \
             else -1 end as col_hit_ratio_percent \
    from table(mon_get_bufferpool(NULL,-2)) mgbp \
    WHERE mgbp.bp_name not like 'IBMSYS%'
    #return(bp_details)
    df = df.append(bp_details)
    bp_details=None
#bp_details
df.columns = ['BP Name', 'Logical Reads', 'Data Hit Ratio', 'Index Hit Ratio', 'XML Hit Ratio', 'Column Hit Ratio']
df

Done.


,BP Name,Logical Reads,Data Hit Ratio,Index Hit Ratio,XML Hit Ratio,Column Hit Ratio
0,IBMDEFAULTBP,67053,99.52,97.54,-1.00,-1.00


## Package Cache

In [17]:
if baselines == 1:
    pkg_cache=%sql select \
        case when pkg_cache_lookups-{mgd_baseline[0].pkg_cache_lookups} > 0 \
            then decimal((1-(float(pkg_cache_inserts-{mgd_baseline[0].pkg_cache_inserts})/float(pkg_cache_lookups-{mgd_baseline[0].pkg_cache_lookups})))*100,5,2)  \
            else -1 end as pkg_cache_hitratio \
        , pkg_cache_num_overflows-{mgd_baseline[0].pkg_cache_num_overflows} as pkg_cache_num_overflows \
        from table(mon_get_database(-2)) as mgd
else:
    pkg_cache=%sql select decimal((1-(float(pkg_cache_inserts)/float(pkg_cache_lookups)))*100,5,2) as pkg_cache_hitratio \
        , pkg_cache_num_overflows \
    from table(mon_get_database(-2)) as mgd
pkg_cache


Done.


pkg_cache_hitratio,pkg_cache_num_overflows
90.08,18


## Catalog Cache

In [18]:
if baselines == 1:
    cat_cache=%sql select \
        case when cat_cache_lookups-{mgd_baseline[0].cat_cache_lookups} > 0 \
            then decimal((1-(float(cat_cache_inserts-{mgd_baseline[0].cat_cache_inserts})/float(cat_cache_lookups-{mgd_baseline[0].pkg_cache_lookups})))*100,5,2)  \
            else -1 end as cat_cache_hitratio \
        , cat_cache_overflows-{mgd_baseline[0].cat_cache_overflows} as cat_cache_overflows \
        from table(mon_get_database(-2)) as mgd
else:
    cat_cache=%sql select decimal((1-(float(cat_cache_inserts)/float(cat_cache_lookups)))*100,5,2) as cat_cache_hitratio \
        , cat_cache_overflows \
    from table(mon_get_database(-2)) as mgd
cat_cache

Done.


cat_cache_hitratio,cat_cache_overflows
99.57,0


## Transaction Logs

In [19]:
if baselines == 1:
    tl_data= %sql select log_reads-{mgtl_baseline[0].log_reads} as log_reads \
    , log_writes-{mgtl_baseline[0].log_writes} as log_writes \
from table(mon_get_transaction_log(-2)) as mgtl 
else:
    tl_data= %sql select log_reads \
        , log_writes \
    from table(mon_get_transaction_log(-2)) as mgtl
tl_data

Done.


log_reads,log_writes
0,2694


In [20]:
%sql select decimal(100*(float(total_log_used)/float(total_log_available)),5,2) as tot_log_use_pct \
        , sec_logs_allocated \
        , sec_log_used_top \
    from table(mon_get_transaction_log(-2)) as mgtl

Done.


tot_log_use_pct,sec_logs_allocated,sec_log_used_top
4.43,0,0


In [21]:
%sql with t1 as (select substr(start_time, 1, 10) as ts \
  ,count(*) as archives \
from sysibmadm.db_history \
where operation = 'X' \
   and start_time > current timestamp - 7 days \
group by substr(start_time, 1, 10) ) \
select avg(archives) as avg_arch_per_hour \
    , max(archives) as max_arch_in_hour \
    , min(archives) as min_arch_in_hour \
from t1 \

Done.


avg_arch_per_hour,max_arch_in_hour,min_arch_in_hour
2,2,2


## Locking

In [22]:
if baselines == 1:
    locking= %sql select deadlocks-{mgd_baseline[0].deadlocks} as deadlocks \
        , lock_timeouts-{mgd_baseline[0].lock_timeouts} as lock_timeouts \
        , lock_escals-{mgd_baseline[0].lock_escals} as lock_escals\
    from table(mon_get_database(-2)) as mgd
else:
     locking= %sql select deadlocks \
        , lock_timeouts \
        , lock_escals \
    from table(mon_get_database(-2)) as mgd
locking

Done.


deadlocks,lock_timeouts,lock_escals
0,0,0


## Sorts

In [23]:
if baselines == 1:
    sorts= %sql select \
        case when total_sorts-{mgd_baseline[0].total_sorts} > 0 \
            then decimal(float(sort_overflows-{mgd_baseline[0].sort_overflows})/float(total_sorts-{mgd_baseline[0].total_sorts}),5,2) \
            else 0 end as sort_overflow_pct \
    from table(mon_get_database(-2)) as mgd
else:
    sorts= %sql select \
        case when total_sorts >0 \
            then decimal(float(sort_overflows)/float(total_sorts),5,2) \
            else 0 end as sort_overflow_pct \
    from table(mon_get_database(-2)) as mgd
sorts

Done.


sort_overflow_pct
0.00
